In [1]:
pip install optuna #best library for hyperparameter tunning

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [65]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [66]:
df=pd.read_csv(r"C:\Users\akshay\Downloads\Admission_Prediction.csv")

In [67]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [68]:
df["GRE Score"]=df['GRE Score'].fillna(df['GRE Score'].mean())
df["TOEFEL Score"]=df['GRE Score'].fillna(df['GRE Score'].mean())
df["University Rating"]=df['GRE Score'].fillna(df['GRE Score'].mean())

In [69]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,TOEFEL Score
count,500.000000,500.000000,490.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000,500.000000
mean,250.500000,316.558763,107.187755,316.558763,3.374000,3.48400,8.576440,0.560000,0.72174,316.558763
std,144.481833,11.103952,6.112899,11.103952,0.991004,0.92545,0.604813,0.496884,0.14114,11.103952
min,1.000000,290.000000,92.000000,290.000000,1.000000,1.00000,6.800000,0.000000,0.34000,290.000000
25%,125.750000,309.000000,103.000000,309.000000,2.500000,3.00000,8.127500,0.000000,0.63000,309.000000
50%,250.500000,316.558763,107.000000,316.558763,3.500000,3.50000,8.560000,1.000000,0.72000,316.558763
75%,375.250000,324.000000,112.000000,324.000000,4.000000,4.00000,9.040000,1.000000,0.82000,324.000000
max,500.000000,340.000000,120.000000,340.000000,5.000000,5.00000,9.920000,1.000000,0.97000,340.000000


In [70]:
x=df.drop(columns=['Serial No.','Chance of Admit'])

In [71]:
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,TOEFEL Score
0,337.000000,118.0,337.000000,4.5,4.5,9.65,1,337.000000
1,324.000000,107.0,324.000000,4.0,4.5,8.87,1,324.000000
2,316.558763,104.0,316.558763,3.0,3.5,8.00,1,316.558763
3,322.000000,110.0,322.000000,3.5,2.5,8.67,1,322.000000
4,314.000000,103.0,314.000000,2.0,3.0,8.21,0,314.000000
...,...,...,...,...,...,...,...,...
495,332.000000,108.0,332.000000,4.5,4.0,9.02,1,332.000000
496,337.000000,117.0,337.000000,5.0,5.0,9.87,1,337.000000
497,330.000000,120.0,330.000000,4.5,5.0,9.56,1,330.000000
498,312.000000,103.0,312.000000,4.0,5.0,8.43,0,312.000000


In [72]:
y=df['Chance of Admit']

In [73]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [74]:
from sklearn.preprocessing import StandardScaler
std_sca= StandardScaler()
x=std_sca.fit_transform(x)

In [75]:
parameter from documents
from sklearn.metrics import mean_squared_error
def objective(trail,data=x,target=y):
    x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=.10,random_state=30)
    param={
        'tree_method':'gpu_hist',
        'lambda':trail.suggest_loguniform('lamba',le-4,10.0),
        'alpha':trail.suggest_loguniform('alpha',le-4,10.0),
        'colsample_bytree':trial.suggest_categorical('colsample_bytree',[.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample':trail.suggest_categorical('subsample',[.3,.4,.5,.6,.7,.8,.9,1]),
        'colsample_bytree':trial.suggest_categorical('learningrate',[.00001,.00003,.008,1,10,20]),
        'n_estimator':30000,
        'max_depth':trail.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
        'random_state':trail.suggest_catagorical('random_state',[10,20,30,2000,3454,243123]),
        'min_child_weight':trial.suggest_int('min_child_weight',1,200),
        'verbosity':3
        
    }
    xgb_reg_model=xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    pred_xgb=xgb_reg_model.predict(test_x)
    rmse=mean_squared_error(test_y,pred_xgb)
    return rmse

In [76]:
find_params=optuna.create_study(direction='minimize')
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params

[I 2023-02-10 18:11:30,587] A new study created in memory with name: no-name-12e21baa-724b-4316-8c7e-8d74c4c581c2
[W 2023-02-10 18:11:30,601] Trial 0 failed with parameters: {} because of the following error: NameError("name 'le' is not defined").
Traceback (most recent call last):
  File "C:\Users\akshay\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/849649137.py", line 6, in objective
    'lambda':trail.suggest_loguniform('lamba',le-4,10.0),
NameError: name 'le' is not defined
[W 2023-02-10 18:11:30,606] Trial 0 failed with value None.


NameError: name 'le' is not defined

In [77]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        'tree_method':'gpu_hist',
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

In [78]:
find_params=optuna.create_study(direction='minimize')
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params

[I 2023-02-10 18:11:35,393] A new study created in memory with name: no-name-c3f426f3-0776-441e-ba73-5930af718103
C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/504234329.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/504234329.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[18:11:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:1.39175
[1]	validation_0-rmse:9.26605
[2]	validation_0-rmse:64.90962
[3]	validation_0-rmse:453.90565
[4]	validation_0-rmse:3177.28459
[5]	validation_0-rmse:22238.94914
[6]	validation_0-rmse:155661.07340
[7]	validation_0-rmse:1089551.71116
[8]	validation_0-rmse:7626261.24513
[9]	validation_0-rmse:53380014.90143
[10]	validation_0-rmse:373634110.80639
[11]	validation_0-rmse:2615255018.12439
[12]	validation_0-rmse:18305482296.79357
[13]	validation_0-rmse:128128845288.83902
[14]	validation_0-rmse:896831971568.41589
[15]	validation_0-rmse:6277309594727.43164
[16]	validation_0-rmse:43938135957810.04688
[17]	validation_0-rmse:307544695833982.93750
[18]	validation_0-rmse:2152667433788680.00000
[19]	validation_0-rmse:15067534684857540.00000
[20]	validation_0-rmse:1054657290248

[I 2023-02-10 18:11:36,911] Trial 0 finished with value: 8.303862686323146e+72 and parameters: {'lambda': 0.014196913209920862, 'alpha': 5.243727508262309, 'colsample_bytree': 0.3, 'subsample': 0.5, 'learning_rate': 8, 'max_depth': 7, 'random_state': 20, 'min_child_weight': 142}. Best is trial 0 with value: 8.303862686323146e+72.


[18:11:36] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.23075
[1]	validation_0-rmse:0.22663
[2]	validation_0-rmse:0.22255
[3]	validation_0-rmse:0.21859
[4]	validation_0-rmse:0.21473


C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/504234329.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/504234329.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



...

[18:11:49] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.08364
[1]	validation_0-rmse:0.07530
[2]	validation_0-rmse:0.07835
[3]	validation_0-rmse:0.07590
[4]	validation_0-rmse:0.07354


C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/504234329.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

C:\Users\akshay\AppData\Local\Temp/ipykernel_40416/504234329.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[5]	validation_0-rmse:0.07281
[6]	validation_0-rmse:0.07112
[7]	validation_0-rmse:0.07181
[8]	validation_0-rmse:0.07036
[9]	validation_0-rmse:0.07035
[10]	validation_0-rmse:0.07157
[11]	validation_0-rmse:0.07070
[12]	validation_0-rmse:0.07180
[13]	validation_0-rmse:0.07430
[14]	validation_0-rmse:0.07299
[15]	validation_0-rmse:0.07265
[16]	validation_0-rmse:0.07253
[17]	validation_0-rmse:0.07259
[18]	validation_0-rmse:0.07310
[19]	validation_0-rmse:0.07203
[20]	validation_0-rmse:0.07263
[21]	validation_0-rmse:0.07457
[22]	validation_0-rmse:0.07273
[23]	validation_0-rmse:0.07280
[24]	validation_0-rmse:0.07241
[25]	validation_0-rmse:0.07132
[26]	validation_0-rmse:0.07264
[27]	validation_0-rmse:0.07213
[28]	validation_0-rmse:0.07198
[29]	validation_0-rmse:0.07233
[30]	validation_0-rmse:0.07146
[31]	validation_0-rmse:0.07348
[32]	validation_0-rmse:0.07243
[33]	validation_0-rmse:0.07325
[34]	validation_0-rmse:0.07219
[35]	validation_0-rmse:0.07267
[36]	validation_0-rmse:0.07165
[37]	validati

[I 2023-02-10 18:11:51,107] Trial 9 finished with value: 0.004798704541338985 and parameters: {'lambda': 0.0021989489904092083, 'alpha': 0.023857162070464837, 'colsample_bytree': 1, 'subsample': 0.8, 'learning_rate': 1, 'max_depth': 5, 'random_state': 20, 'min_child_weight': 64}. Best is trial 9 with value: 0.004798704541338985.


{'lambda': 0.0021989489904092083,
 'alpha': 0.023857162070464837,
 'colsample_bytree': 1,
 'subsample': 0.8,
 'learning_rate': 1,
 'max_depth': 5,
 'random_state': 20,
 'min_child_weight': 64}

In [79]:
{'lambda': 0.009664478610044897,
 'alpha': 1.212494550192719,
 'colsample_bytree': 0.8,
 'subsample': 0.8,
 'learning_rate': 0.008,
 'max_depth': 7,
 'random_state': 10,
 'min_child_weight': 5}

{'lambda': 0.009664478610044897,
 'alpha': 1.212494550192719,
 'colsample_bytree': 0.8,
 'subsample': 0.8,
 'learning_rate': 0.008,
 'max_depth': 7,
 'random_state': 10,
 'min_child_weight': 5}

In [80]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,8.303863e+72,2023-02-10 18:11:35.400726,2023-02-10 18:11:36.908279,0 days 00:00:01.507553,5.243728,0.3,0.014197,8.0000,7,142,20,0.5,COMPLETE
1,1,5.930147e-03,2023-02-10 18:11:36.916333,2023-02-10 18:11:38.548146,0 days 00:00:01.631813,0.999551,0.3,0.002570,0.0200,8,81,243123,0.9,COMPLETE
2,2,9.028580e+72,2023-02-10 18:11:38.555145,2023-02-10 18:11:40.229181,0 days 00:00:01.674036,0.001159,0.6,0.350607,8.0000,10,66,10,0.8,COMPLETE
3,3,6.119619e-03,2023-02-10 18:11:40.237178,2023-02-10 18:11:41.795127,0 days 00:00:01.557949,0.010214,0.4,0.015059,1.0000,5,101,10,0.9,COMPLETE
4,4,5.270112e-02,2023-02-10 18:11:41.801393,2023-02-10 18:11:43.224599,0 days 00:00:01.423206,0.009368,0.4,0.139820,0.0003,5,173,243123,0.5,COMPLETE
5,5,1.433031e-02,2023-02-10 18:11:43.230957,2023-02-10 18:11:45.139885,0 days 00:00:01.908928,0.015691,0.9,0.000166,0.0080,12,75,243123,0.9,COMPLETE
6,6,1.520383e-02,2023-02-10 18:11:45.146884,2023-02-10 18:11:46.671550,0 days 00:00:01.524666,1.622432,0.4,7.015763,0.0100,12,127,2000,0.7,COMPLETE
7,7,1.789379e-02,2023-02-10 18:11:46.677548,2023-02-10 18:11:47.955095,0 days 00:00:01.277547,0.520041,0.8,0.008952,0.0200,3,183,243123,0.7,COMPLETE
8,8,2.043880e-02,2023-02-10 18:11:47.961095,2023-02-10 18:11:49.330904,0 days 00:00:01.369809,0.085355,0.3,0.022068,0.0100,5,55,2000,0.2,COMPLETE
9,9,4.798705e-03,2023-02-10 18:11:49.336902,2023-02-10 18:11:51.105437,0 days 00:00:01.768535,0.023857,1.0,0.002199,1.0000,5,64,20,0.8,COMPLETE


In [81]:
!NVIDIA-SMI #TO KNOW GPU SO IN CASE OF TWO GPU U CAN GIVE ID

Invalid combination of input arguments. Please run 'nvidia-smi -h' for help.



In [82]:
optuna.visualization.plot_optimization_history(find_params)

In [83]:
optuna.visualization.plot_slice(find_params)

In [84]:
optuna.visualization.plot_contour(find_params,params=['alpha','lambda'])

In [85]:
best_params={'lambda': 0.009664478610044897,
 'alpha': 1.212494550192719,
 'colsample_bytree': 0.8,
 'subsample': 0.8,
 'learning_rate': 0.008,
 'max_depth': 7,
 'random_state': 10,
 'min_child_weight': 5}

In [86]:
xgb_final_model=xgb.XGBRegressor(**best_params)

In [87]:
xgb_final_model.fit(x,y)

XGBRegressor(alpha=1.212494550192719, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.009664478610044897,
             learning_rate=0.008, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=5, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [88]:
xgb_final_model.score(x,y)

0.044422148939256